In [1]:
import os

# Set the environment variables before using the transformers library
os.environ["HF_HOME"] = "/serenity/scratch/hkolisetty6/.cache/huggingface"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

import sys
sys.path.insert(0, "./transformers/src")
sys.path.insert(0, "./peft/src")

In [2]:
from profiler import (
    parse_args,
    get_last_checkpoint,
    load_model,
    set_width_mask_and_bias,
    make_data_module,
    Seq2SeqTrainer,
    profile_latencies,
    get_latency_stats,
    logging,
)

/serenity/scratch/dgarg/anaconda3/envs/amoaballm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Profiling with source_max_length of 64
- Depths: 26, 27

In [ ]:
args = [
    "--model_name_or_path", "meta-llama/Llama-2-7b-hf",
    "--output_dir", "amoeba_llama2",
    "--do_predict", "True",
    "--do_eval", "False",
    "--do_train", "False",
    "--do_mmlu_eval", "False",
    "--enable_shrinking",
    "--min_num_layer", "20",
    "--shrinking_method", "calib_dp",
    "--shrinking_file", "dp_selection_strategy.npy",
    "--shrinkable_width",
    "--width_choice", "[1,7/8,3/4,5/8,1/2]",
    "--prune_width_method", "flap",
    "--use_moe_lora",
    "--moe_num_expert", "5",
    "--moe_topk", "2",
    "--eval_num_layer", "32",
    "--eval_num_width", "1",
    "--predict_with_generate", "True",
    "--source_max_len", "64",
    "--max_new_tokens", "128",
]
args, training_args = parse_args(args_list=args)
checkpoint_dir = get_last_checkpoint(args.output_dir)
model, tokenizer = load_model(args, checkpoint_dir)
set_width_mask_and_bias(model, args)
logger = logging.getLogger(__name__)
results = {}

/serenity/scratch/dgarg/anaconda3/envs/amoaballm/lib/python3.9/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.25s/it]


Adding correct special tokens to the llama tokenizer
Loading adapters from checkpoint


In [4]:
args.eval_num_width = 1.0
args.eval_num_layer = 26
key = f'l{args.eval_num_layer}w{args.eval_num_width}sml{args.source_max_len}mnt128'
results[key] = []
ttfts = []
tbts = []

for bs in [1, 2, 4]:
    training_args.per_device_eval_batch_size = bs
    args.eval_dataset_size = training_args.per_device_eval_batch_size * 50 + 1
    data_module = make_data_module(tokenizer=tokenizer, args=args)
    trainer = Seq2SeqTrainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        **{k: v for k, v in data_module.items() if k != "predict_dataset"},
    )
    ttft, tbt = profile_latencies(model, tokenizer, args, logger, trainer, data_module)
    ttfts.append(ttft)
    tbts.append(tbt)
    results[key].append(get_latency_stats(ttft, tbt, bs))

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Splitting train dataset in train and validation according to `eval_dataset_size`
input_ids shape: torch.Size([1, 65])


input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
i

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Splitting train dataset in train and validation according to `eval_dataset_size`
input_ids shape: torch.Size([2, 65])


input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
i

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Splitting train dataset in train and validation according to `eval_dataset_size`
input_ids shape: torch.Size([4, 65])


input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
i

In [5]:
import json
with open(f'latency_results_{key}.json', 'w') as fout:
    json.dump(results, fout)
with open(f'latencies_{key}.json', 'w') as fout:
    json.dump({'ttft': ttfts, 'tbt': tbts}, fout)

In [6]:
args.eval_num_width = 1.0
args.eval_num_layer = 27
key = f'l{args.eval_num_layer}w{args.eval_num_width}sml{args.source_max_len}mnt128'
results = {}
results[key] = []
ttfts = []
tbts = []

for bs in [1, 2, 4]:
    training_args.per_device_eval_batch_size = bs
    args.eval_dataset_size = training_args.per_device_eval_batch_size * 50 + 1
    data_module = make_data_module(tokenizer=tokenizer, args=args)
    trainer = Seq2SeqTrainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        **{k: v for k, v in data_module.items() if k != "predict_dataset"},
    )
    ttft, tbt = profile_latencies(model, tokenizer, args, logger, trainer, data_module)
    ttfts.append(ttft)
    tbts.append(tbt)
    results[key].append(get_latency_stats(ttft, tbt, bs))

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Splitting train dataset in train and validation according to `eval_dataset_size`
input_ids shape: torch.Size([1, 65])


input_ids shape: torch.Size([1, 65])


input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
input_ids shape: torch.Size([1, 65])
i

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Splitting train dataset in train and validation according to `eval_dataset_size`
input_ids shape: torch.Size([2, 65])


input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
input_ids shape: torch.Size([2, 65])
i

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Splitting train dataset in train and validation according to `eval_dataset_size`
input_ids shape: torch.Size([4, 65])


input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
input_ids shape: torch.Size([4, 65])
i

In [7]:
import json
with open(f'latency_results_{key}.json', 'w') as fout:
    json.dump(results, fout)
with open(f'latencies_{key}.json', 'w') as fout:
    json.dump({'ttft': ttfts, 'tbt': tbts}, fout)

## Profiling with source_max_length of 128
- Depths: 26, 27

In [3]:
args = [
    "--model_name_or_path", "meta-llama/Llama-2-7b-hf",
    "--output_dir", "amoeba_llama2",
    "--do_predict", "True",
    "--do_eval", "False",
    "--do_train", "False",
    "--do_mmlu_eval", "False",
    "--enable_shrinking",
    "--min_num_layer", "20",
    "--shrinking_method", "calib_dp",
    "--shrinking_file", "dp_selection_strategy.npy",
    "--shrinkable_width",
    "--width_choice", "[1,7/8,3/4,5/8,1/2]",
    "--prune_width_method", "flap",
    "--use_moe_lora",
    "--moe_num_expert", "5",
    "--moe_topk", "2",
    "--eval_num_layer", "32",
    "--eval_num_width", "1",
    "--predict_with_generate", "True",
    "--source_max_len", "128",
    "--max_new_tokens", "128",
]
args, training_args = parse_args(args_list=args)
checkpoint_dir = get_last_checkpoint(args.output_dir)
model, tokenizer = load_model(args, checkpoint_dir)
set_width_mask_and_bias(model, args)
logger = logging.getLogger(__name__)
results = {}

/serenity/scratch/dgarg/anaconda3/envs/amoaballm/lib/python3.9/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.17s/it]


Adding correct special tokens to the llama tokenizer
Loading adapters from checkpoint


In [4]:
args.eval_num_width = 1.0
args.eval_num_layer = 26
key = f'l{args.eval_num_layer}w{args.eval_num_width}sml{args.source_max_len}mnt128'
results = {}
results[key] = []
ttfts = []
tbts = []

for bs in [1, 2, 4]:
    training_args.per_device_eval_batch_size = bs
    args.eval_dataset_size = training_args.per_device_eval_batch_size * 70 + 1
    data_module = make_data_module(tokenizer=tokenizer, args=args)
    trainer = Seq2SeqTrainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        **{k: v for k, v in data_module.items() if k != "predict_dataset"},
    )
    ttft, tbt = profile_latencies(model, tokenizer, args, logger, trainer, data_module)
    ttfts.append(ttft)
    tbts.append(tbt)
    results[key].append(get_latency_stats(ttft, tbt, bs))

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Splitting train dataset in train and validation according to `eval_dataset_size`
input_ids shape: torch.Size([1, 129])


input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids sh

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Splitting train dataset in train and validation according to `eval_dataset_size`
input_ids shape: torch.Size([2, 129])


input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids sh

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Splitting train dataset in train and validation according to `eval_dataset_size`
input_ids shape: torch.Size([4, 129])


input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids sh

In [5]:
import json
with open(f'latency_results_{key}.json', 'w') as fout:
    json.dump(results, fout)
with open(f'latencies_{key}.json', 'w') as fout:
    json.dump({'ttft': ttfts, 'tbt': tbts}, fout)

In [6]:
args.eval_num_width = 1.0
args.eval_num_layer = 27
key = f'l{args.eval_num_layer}w{args.eval_num_width}sml{args.source_max_len}mnt128'
results = {}
results[key] = []
ttfts = []
tbts = []

for bs in [1, 2, 4]:
    training_args.per_device_eval_batch_size = bs
    args.eval_dataset_size = training_args.per_device_eval_batch_size * 70 + 1
    data_module = make_data_module(tokenizer=tokenizer, args=args)
    trainer = Seq2SeqTrainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        **{k: v for k, v in data_module.items() if k != "predict_dataset"},
    )
    ttft, tbt = profile_latencies(model, tokenizer, args, logger, trainer, data_module)
    ttfts.append(ttft)
    tbts.append(tbt)
    results[key].append(get_latency_stats(ttft, tbt, bs))

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Splitting train dataset in train and validation according to `eval_dataset_size`
input_ids shape: torch.Size([1, 129])


input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids sh

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Splitting train dataset in train and validation according to `eval_dataset_size`
input_ids shape: torch.Size([2, 129])


input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids sh

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Splitting train dataset in train and validation according to `eval_dataset_size`
input_ids shape: torch.Size([4, 129])


input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids sh

In [7]:
import json
with open(f'latency_results_{key}.json', 'w') as fout:
    json.dump(results, fout)
with open(f'latencies_{key}.json', 'w') as fout:
    json.dump({'ttft': ttfts, 'tbt': tbts}, fout)

# Profiling with source_max_len of 128 and max_new_tokens of 256

In [3]:
args = [
    "--model_name_or_path", "meta-llama/Llama-2-7b-hf",
    "--output_dir", "amoeba_llama2",
    "--do_predict", "True",
    "--do_eval", "False",
    "--do_train", "False",
    "--do_mmlu_eval", "False",
    "--enable_shrinking",
    "--min_num_layer", "20",
    "--shrinking_method", "calib_dp",
    "--shrinking_file", "dp_selection_strategy.npy",
    "--shrinkable_width",
    "--width_choice", "[1,7/8,3/4,5/8,1/2]",
    "--prune_width_method", "flap",
    "--use_moe_lora",
    "--moe_num_expert", "5",
    "--moe_topk", "2",
    "--eval_num_layer", "32",
    "--eval_num_width", "1",
    "--predict_with_generate", "True",
    "--source_max_len", "128",
    "--max_new_tokens", "256",
]
args, training_args = parse_args(args_list=args)
checkpoint_dir = get_last_checkpoint(args.output_dir)
model, tokenizer = load_model(args, checkpoint_dir)
set_width_mask_and_bias(model, args)
logger = logging.getLogger(__name__)
results = {}

/serenity/scratch/dgarg/anaconda3/envs/amoaballm/lib/python3.9/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.51s/it]


Adding correct special tokens to the llama tokenizer
Loading adapters from checkpoint


In [4]:
args.eval_num_width = 1.0
args.eval_num_layer = 26
key = f'l{args.eval_num_layer}w{args.eval_num_width}sml{args.source_max_len}mnt256'
results[key] = []
ttfts = []
tbts = []

for bs in [1, 2, 4]:
    training_args.per_device_eval_batch_size = bs
    args.eval_dataset_size = training_args.per_device_eval_batch_size * 70 + 1
    data_module = make_data_module(tokenizer=tokenizer, args=args)
    trainer = Seq2SeqTrainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        **{k: v for k, v in data_module.items() if k != "predict_dataset"},
    )
    ttft, tbt = profile_latencies(model, tokenizer, args, logger, trainer, data_module)
    ttfts.append(ttft)
    tbts.append(tbt)
    results[key].append(get_latency_stats(ttft, tbt, bs))

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Splitting train dataset in train and validation according to `eval_dataset_size`
input_ids shape: torch.Size([1, 129])


input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids sh

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Splitting train dataset in train and validation according to `eval_dataset_size`
input_ids shape: torch.Size([2, 129])


input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids sh

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Splitting train dataset in train and validation according to `eval_dataset_size`
input_ids shape: torch.Size([4, 129])


input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids sh

In [5]:
import json
with open(f'latency_results_{key}.json', 'w') as fout:
    json.dump(results, fout)
with open(f'latencies_{key}.json', 'w') as fout:
    json.dump({'ttft': ttfts, 'tbt': tbts}, fout)

In [6]:
args.eval_num_width = 1.0
args.eval_num_layer = 27
key = f'l{args.eval_num_layer}w{args.eval_num_width}sml{args.source_max_len}mnt256'
results = {}
results[key] = []
ttfts = []
tbts = []

for bs in [1, 2, 4]:
    training_args.per_device_eval_batch_size = bs
    args.eval_dataset_size = training_args.per_device_eval_batch_size * 70 + 1
    data_module = make_data_module(tokenizer=tokenizer, args=args)
    trainer = Seq2SeqTrainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        **{k: v for k, v in data_module.items() if k != "predict_dataset"},
    )
    ttft, tbt = profile_latencies(model, tokenizer, args, logger, trainer, data_module)
    ttfts.append(ttft)
    tbts.append(tbt)
    results[key].append(get_latency_stats(ttft, tbt, bs))

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Splitting train dataset in train and validation according to `eval_dataset_size`
input_ids shape: torch.Size([1, 129])


input_ids shape: torch.Size([1, 129])


input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids shape: torch.Size([1, 129])
input_ids sh

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Splitting train dataset in train and validation according to `eval_dataset_size`
input_ids shape: torch.Size([2, 129])


input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids shape: torch.Size([2, 129])
input_ids sh

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Splitting train dataset in train and validation according to `eval_dataset_size`
input_ids shape: torch.Size([4, 129])


input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids shape: torch.Size([4, 129])
input_ids sh

In [7]:
import json
with open(f'latency_results_{key}.json', 'w') as fout:
    json.dump(results, fout)
with open(f'latencies_{key}.json', 'w') as fout:
    json.dump({'ttft': ttfts, 'tbt': tbts}, fout)